In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver


In [2]:
load_dotenv()

llm = ChatOpenAI()

In [3]:
class JokeState(TypedDict) :
    topic : str
    joke: str
    explanation : str

In [4]:
def generate_joke(state: JokeState) :
      
      prompt = f'generate a joke on the topic {state['topic']}'
      response = llm.invoke(prompt).content

      return {'joke' : response}

In [5]:
def explain_joke(state: JokeState) :
      
      prompt = f'explain the following joke on the topic-{state['topic']}, joke-{state['joke']}'
      response = llm.invoke(prompt).content
      return {'explanation' : response}

In [8]:
graph = StateGraph(JokeState) 
graph.add_node('joke', generate_joke)
graph.add_node('explanation', explain_joke)

graph.add_edge(START, 'joke')
graph.add_edge('joke','explanation')
graph.add_edge('explanation', END)

checkpointer = InMemorySaver() # this will save all the intermediate state of the workflow

workflow = graph.compile(checkpointer=checkpointer)

In [12]:
config1 = {"configurable": {'thread_id' : 1}} # in order to save the invoke in some particular thread with id 1 
workflow.invoke({"topic":  "dad joke"}, config=config1)


{'topic': 'dad joke',
 'joke': "Why did the dad joke go to therapy? Because it couldn't stop cracking itself up!",
 'explanation': "This joke plays on the stereotype that dad jokes are extremely corny and often bring a lot of laughter to the dad telling them. In this case, the dad joke is portrayed as being so amused by its own joke that it couldn't stop laughing, leading it to seek therapy to address its self-amusement. It's a playful way to poke fun at the cheesy and sometimes cringe-worthy nature of dad jokes."}

In [13]:
workflow.get_state(config1) # this will show the final value of the state at the end

StateSnapshot(values={'topic': 'dad joke', 'joke': "Why did the dad joke go to therapy? Because it couldn't stop cracking itself up!", 'explanation': "This joke plays on the stereotype that dad jokes are extremely corny and often bring a lot of laughter to the dad telling them. In this case, the dad joke is portrayed as being so amused by its own joke that it couldn't stop laughing, leading it to seek therapy to address its self-amusement. It's a playful way to poke fun at the cheesy and sometimes cringe-worthy nature of dad jokes."}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c11e5-cda2-6394-8006-96521a4d12b9'}}, metadata={'source': 'loop', 'step': 6, 'parents': {}}, created_at='2025-11-14T05:54:19.946485+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c11e5-c199-60e5-8005-b0471009fba7'}}, tasks=(), interrupts=())

In [14]:
list(workflow.get_state_history(config1)) # this will give the intermediate state value 

[StateSnapshot(values={'topic': 'dad joke', 'joke': "Why did the dad joke go to therapy? Because it couldn't stop cracking itself up!", 'explanation': "This joke plays on the stereotype that dad jokes are extremely corny and often bring a lot of laughter to the dad telling them. In this case, the dad joke is portrayed as being so amused by its own joke that it couldn't stop laughing, leading it to seek therapy to address its self-amusement. It's a playful way to poke fun at the cheesy and sometimes cringe-worthy nature of dad jokes."}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c11e5-cda2-6394-8006-96521a4d12b9'}}, metadata={'source': 'loop', 'step': 6, 'parents': {}}, created_at='2025-11-14T05:54:19.946485+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c11e5-c199-60e5-8005-b0471009fba7'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'dad joke', 'joke': "Why did the dad